In [164]:
import nibabel as nib
import pydicom
import numpy as np
import os
from pydicom.dataset import Dataset, FileDataset
from datetime import datetime
from pydicom.uid import generate_uid
import matplotlib.pyplot as plt
import requests
import simple_orthanc


# TO make it faster store study id!!!!!
"""
def delete_patient_studies( patient_to_delete, study_to_delete, orthanc_url='http://localhost:8042'):
    # Retrieve patient details to get the studies
    patient_url = f"{orthanc_url}/patients/"
    patient_response = requests.get(patient_url)

    if patient_response.status_code != 200:
        print(f"Failed to retrieve patient information. Status code: {patient_response.status_code}")
        print(f"Response: {patient_response.text}")
        return

    patients = patient_response.json()
    
    for patient_id in patients:
        patient_url = f"{orthanc_url}/patients/{patient_id}"
        patient_response = requests.get(patient_url)

        if patient_response.json().get('MainDicomTags', {}).get('PatientID')== patient_to_delete:
            studies = patient_response.json().get('Studies', [])
            
            for study in studies:
                study_url= f"{orthanc_url}/studies/{study}"
                study_response = requests.get(study_url)
                if study_response.json().get('MainDicomTags', {}).get('StudyID')==study_to_delete:
                    
                    delete_response = requests.delete(study_url)
                    print(f"Deleted Study: {delete_response.json()}")
            return
"""

def delete_series(study_to_delete, orthanc_url='http://localhost:8042'):
    series_url = f"{orthanc_url}/series/"
    params = {}
    params['expand'] = True
    params['requestedTags'] = "StudyInstanceUID"

    
    series_response = requests.get(series_url, params)
    series_id = [entry['ID'] for entry in series_response.json() if entry.get('RequestedTags', {}).get('StudyInstanceUID') == study_to_delete]
    for serie_id in series_id:
        serie_response = requests.delete(f"{orthanc_url}/series/"+serie_id)
        if serie_response.status_code != 200:
            print(f"Failed to retrieve patient information. Status code: {serie_response.status_code}")
            print(f"Response: {serie_response.text}")
            return
        print(f"Deleted Study: {serie_id}")


def upload_dicom_folder(dicom_folder):
    # Initialize the Orthanc client
    orthanc = simple_orthanc.Orthanc()

    
    orthanc.upload_folder(dicom_folder, test_dicom=True, recursive=True)

def nifti_to_dicom(nifti_file,
    output_folder,
    series_description,
    series_instance_uid=generate_uid(),
    series_number="1",
    reference_dicom_file = "../data/dicom/real/extensive-consolidation-v2-sampl/image.0001.dcm",
    modality='CT',
    study_instance_uid = '1.3.6.1.4.1.14519.5.2.1.6834.5010.119076582718824960697635220329', # should be different for AI/non-AI
    study_id='1', # should be different for AI/non-AI
    accession_number = '1', # should be the same for AI/non-AI
    patient_name="Generative AI Patient",
    patient_id="MedSyn",
    description="",

    ):
    rotate=""
    if modality=='AI':
        content_date=""
        rotate="counterclockwise"
    else:
        content_date = str(datetime.now().date()).replace('-', '')
        rotate="clockwise"

    # Load the NIfTI file
    img = nib.load(nifti_file)
    data = img.get_fdata()
    affine = img.affine

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize common dataset attributes
    ds = pydicom.dcmread(reference_dicom_file)
    ds.PatientName = patient_name
    ds.PatientID = patient_id
    ds.AccessionNumber = accession_number
    ds.StudyInstanceUID = study_instance_uid
    ds.SeriesInstanceUID = series_instance_uid
    ds.SeriesDescription= series_description
    ds.Modality = modality
    ds.SeriesNumber = series_number
    ds.StudyID = study_id
    ds.StudyDescription = description
    ds.Manufacturer = "PythonDicomConversion"
    ds.Rows, ds.Columns = data.shape[:2]
    ds.SliceThickness = float(affine[2, 2])

    ds.SamplesPerPixel = 1
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.HighBit = 15
    ds.PixelRepresentation = 0  # 1 means signed integers




    # Set additional metadata
    ds.ContentDate = content_date
    ds.ContentTime = ""

    ds.StudyDate = ds.ContentDate
    ds.StudyTime = ds.ContentTime
    ds.PatientSex = "O"
    ds.PatientBirthDate = "19000101"


    # Scale pixel data if necessary (e.g., to avoid issues with pixel value ranges)
    data = data - np.min(data)
    data = data / np.max(data) * (2**16 - 1)
    data = data.astype('uint16')

    # reverse in 3rd axis
    #data = data[:,:,::-1]
    # Rotate each slice to the left (90 degrees counterclockwise)
    if rotate == "counterclockwise":
        data = np.rot90(data, k=1, axes=(0, 1))
    elif rotate == "clockwise":
        data = np.rot90(data, k=3, axes=(0, 1))
    
    #print(data)
    # Iterate over each slice and update the dataset
    for i in range(data.shape[2]):
        slice_data = data[:, :, -(i+1)]

        # Update slice-specific attributes
        ds.SOPInstanceUID= generate_uid()
        ds.InstanceNumber = i + 1
        ds.ImagePositionPatient = [0,0,-i] 
        ds.SliceLocation = i * ds.SliceThickness



        # Convert pixel data to the appropriate type and flatten the array
        ds.PixelData = slice_data.tobytes()

        # Visualize the slice
        # plt.imshow(slice_data, cmap='gray')
        # plt.title(f'Slice {i}')
        # plt.show()

        # Save the DICOM file
        dicom_filename = os.path.join(output_folder, f"slice_{i:03d}.dcm")
        ds.save_as(dicom_filename)

    print(f"Conversion complete. DICOM files are saved in {output_folder}")

    # delete if there is already a study for this patient
    delete_series(study_to_delete=study_instance_uid)
    
    upload_dicom_folder(output_folder)
    print("Files Uploaded to Orthanc Server Localhost")


# CT-Rate


## Example 1
- Arterial wall calcification
- Atelectasis
- Lung opacity
- Consolidation

### Original


In [147]:
findings = "Multiple venous collaterals are present in the anterior left chest wall and are associated with the anterior jugular vein at the level of the right sternoclavicular junction. Left subclavian vein collapsed (chronic occlusion pathology?). Trachea, both main bronchi are open. Calcific plaques are observed in the aortic arch. Other mediastinal main vascular structures, heart contour, size are normal. Thoracic aorta diameter is normal. Pericardial effusion-thickening was not observed. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. No enlarged lymph nodes in prevascular, pre-paratracheal, subcarinal or bilateral hilar-axillary pathological dimensions were detected. When examined in the lung parenchyma window; Linear atelectasis is present in both lung parenchyma. Subsegmental atelectasis is observed in the right middle lobe. Thickening of the bronchial wall and peribronchial budding tree-like reticulonodular densities are observed in the bilateral lower lobes. Peribronchial minimal consolidation is seen in the lower lobes in places. The findings were evaluated primarily in favor of the infectious process. The left kidney partially entering the section is atrophic. The right kidney could not be evaluated because it did not enter the section. Other upper abdominal organs included in the sections are normal. No space-occupying lesion was detected in the liver that entered the cross-sectional area. Bilateral adrenal glands were normal and no space-occupying lesion was detected. There are osteophytes with anterior extension in the thoracic vertebrae."
impressions="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/train/train_1_a_1.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1_a_1/original/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description="", 
               modality = "CT",
               study_instance_uid='1.1.1',
               study_id='1', 
               accession_number='1.1',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
               )


c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (69) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1_a_1/original/
Deleted Study: cbd95818-3fdbd45b-4d0d23c4-b9447919-5f4f9983
Files Uploaded to Orthanc Server Localhost


### Generated

In [165]:
findings = "Multiple venous collaterals are present in the anterior left chest wall and are associated with the anterior jugular vein at the level of the right sternoclavicular junction. Left subclavian vein collapsed (chronic occlusion pathology?). Trachea, both main bronchi are open. Calcific plaques are observed in the aortic arch. Other mediastinal main vascular structures, heart contour, size are normal. Thoracic aorta diameter is normal. Pericardial effusion-thickening was not observed. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. No enlarged lymph nodes in prevascular, pre-paratracheal, subcarinal or bilateral hilar-axillary pathological dimensions were detected. When examined in the lung parenchyma window; Linear atelectasis is present in both lung parenchyma. Subsegmental atelectasis is observed in the right middle lobe. Thickening of the bronchial wall and peribronchial budding tree-like reticulonodular densities are observed in the bilateral lower lobes. Peribronchial minimal consolidation is seen in the lower lobes in places. The findings were evaluated primarily in favor of the infectious process. The left kidney partially entering the section is atrophic. The right kidney could not be evaluated because it did not enter the section. Other upper abdominal organs included in the sections are normal. No space-occupying lesion was detected in the liver that entered the cross-sectional area. Bilateral adrenal glands were normal and no space-occupying lesion was detected. There are osteophytes with anterior extension in the thoracic vertebrae."
impressions="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1_a_1/generated/train_1_a_1_sample_0/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.1.2',
               study_id='2', 
               accession_number='1.1',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1_a_1/generated/train_1_a_1_sample_0/
Deleted Study: baed326d-81c82016-ec4dfe9b-f42e3e55-4011b72b
Files Uploaded to Orthanc Server Localhost


In [149]:
findings = "Multiple venous collaterals are present in the anterior left chest wall and are associated with the anterior jugular vein at the level of the right sternoclavicular junction. Left subclavian vein collapsed (chronic occlusion pathology?). Trachea, both main bronchi are open. Calcific plaques are observed in the aortic arch. Other mediastinal main vascular structures, heart contour, size are normal. Thoracic aorta diameter is normal. Pericardial effusion-thickening was not observed. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. No enlarged lymph nodes in prevascular, pre-paratracheal, subcarinal or bilateral hilar-axillary pathological dimensions were detected. When examined in the lung parenchyma window; Linear atelectasis is present in both lung parenchyma. Subsegmental atelectasis is observed in the right middle lobe. Thickening of the bronchial wall and peribronchial budding tree-like reticulonodular densities are observed in the bilateral lower lobes. Peribronchial minimal consolidation is seen in the lower lobes in places. The findings were evaluated primarily in favor of the infectious process. The left kidney partially entering the section is atrophic. The right kidney could not be evaluated because it did not enter the section. Other upper abdominal organs included in the sections are normal. No space-occupying lesion was detected in the liver that entered the cross-sectional area. Bilateral adrenal glands were normal and no space-occupying lesion was detected. There are osteophytes with anterior extension in the thoracic vertebrae."
impressions="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_1.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1_a_1/generated/train_1_a_1_sample_1/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.1.3',
               study_id='3', 
               accession_number='1.1',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1_a_1/generated/train_1_a_1_sample_1/
Deleted Study: bd07d4c9-4baad1fd-308a16e1-fd7a7797-6fea3143
Files Uploaded to Orthanc Server Localhost


In [150]:
findings = "Multiple venous collaterals are present in the anterior left chest wall and are associated with the anterior jugular vein at the level of the right sternoclavicular junction. Left subclavian vein collapsed (chronic occlusion pathology?). Trachea, both main bronchi are open. Calcific plaques are observed in the aortic arch. Other mediastinal main vascular structures, heart contour, size are normal. Thoracic aorta diameter is normal. Pericardial effusion-thickening was not observed. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. No enlarged lymph nodes in prevascular, pre-paratracheal, subcarinal or bilateral hilar-axillary pathological dimensions were detected. When examined in the lung parenchyma window; Linear atelectasis is present in both lung parenchyma. Subsegmental atelectasis is observed in the right middle lobe. Thickening of the bronchial wall and peribronchial budding tree-like reticulonodular densities are observed in the bilateral lower lobes. Peribronchial minimal consolidation is seen in the lower lobes in places. The findings were evaluated primarily in favor of the infectious process. The left kidney partially entering the section is atrophic. The right kidney could not be evaluated because it did not enter the section. Other upper abdominal organs included in the sections are normal. No space-occupying lesion was detected in the liver that entered the cross-sectional area. Bilateral adrenal glands were normal and no space-occupying lesion was detected. There are osteophytes with anterior extension in the thoracic vertebrae."
impressions="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Arterial wall calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_2.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1_a_1/generated/train_1_a_1_sample_2/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.1.4',
               study_id='4', 
               accession_number='1.1',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1_a_1/generated/train_1_a_1_sample_2/
Deleted Study: b4803e76-c2d66cc0-94e32890-8e7f71c4-dd405cce
Files Uploaded to Orthanc Server Localhost


## Example 2
- Arterial wall calcification
- Coronary artery wall calcification
- Lymphadenopathy
- Lung nodule
- Lung opacity


### Original 

In [151]:

findings = "Trachea, both main bronchi are open. Mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Calcific plaques are present in the coronary arteries. Thoracic esophagus calibration was normal and no significant pathological wall thickening was detected. Millimetric lymph nodes with a short axis not exceeding 1 cm are observed in the mediastinum. When examined in the lung parenchyma window; Widespread ground glass densities are observed in both lung parenchyma showing a tendency to central and peripheral fusion. A 5 mm calcific nodule was observed in the posterobasal region of the lower lobe of the right lung. A millimetric hypodense lesion was observed in segment 2 of the liver in the upper abdominal organs included in the sections. Bone structures in the study area are natural. Osteodegenerative changes are observed in the vertebrae."
impressions=" Findings compatible with Covid pneumonia  Aortic and coronary artery atherosclerosis  Hypodense lesion (cyst?) in liver segment 2"
description= "Arterial wall calcification, Coronary artery wall calcification, Lymphadenopathy, Lung nodule, Lung opacity"
nifti_file = '../data/nifti/ct-rate/train/train_10_a_1.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_10_a_1/original/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description="", 
               modality = "CT",
               study_instance_uid='1.2.1',
               study_id='1', 
               accession_number='1.2',
               patient_name="Example Patient 2",
               patient_id="train_10_a_1",
               description= description,

               )


c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (107) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_10_a_1/original/
Deleted Study: e94cf2d2-7113de91-fd2e8a3f-86f8beb5-87c8aeaf
Files Uploaded to Orthanc Server Localhost


### Generated

In [152]:
findings = "Trachea, both main bronchi are open. Mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Calcific plaques are present in the coronary arteries. Thoracic esophagus calibration was normal and no significant pathological wall thickening was detected. Millimetric lymph nodes with a short axis not exceeding 1 cm are observed in the mediastinum. When examined in the lung parenchyma window; Widespread ground glass densities are observed in both lung parenchyma showing a tendency to central and peripheral fusion. A 5 mm calcific nodule was observed in the posterobasal region of the lower lobe of the right lung. A millimetric hypodense lesion was observed in segment 2 of the liver in the upper abdominal organs included in the sections. Bone structures in the study area are natural. Osteodegenerative changes are observed in the vertebrae."
impressions=" Findings compatible with Covid pneumonia  Aortic and coronary artery atherosclerosis  Hypodense lesion (cyst?) in liver segment 2"
description= "Arterial wall calcification, Coronary artery wall calcification, Lymphadenopathy, Lung nodule, Lung opacity"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_10_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_10_a_1/generated/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.2.2',
               study_id='2', 
               accession_number='1.2',
               patient_name="Example Patient 2",
               patient_id="train_10_a_1",
               description= description,

               )


c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (130) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_10_a_1/generated/
Deleted Study: 9b7f98bf-7ed15aba-acea0869-84845525-7575de6c
Files Uploaded to Orthanc Server Localhost


# Example 3
- (Medical material)
- Pericardial effusion
- Emphysema
- Atelectasis
- Lung nodule
- Pleural effusion
- Consolidation


### Original 

In [153]:

findings = "A central venous catheter inserted from the right was observed. Trachea and both main bronchi are open and no obstructive pathology is detected. Mediastinal vascular structures and cardiac examination could not be evaluated optimally due to the lack of contrast. The pulmonary trunk caliber was measured at 30 mm and was wider than normal. Heart contour, the size is natural. There is minimal pericardial effusion. The effusion measuring approximately 80 mm in size is observed on the right. There is an area of increased density in the lung parenchyma adjacent to the effusion, which is considered secondary to compressive atelectasis. No pathological increase in wall thickness was observed in the thoracic esophagus. In the mediastinum, in both axillary regions and in the supraclavicular fossa, no lymph nodes were observed in pathological size and appearance. In the lower lobe of the left lung, there is an area of increase in density consistent with the consolidation observed in the air bronchograms. Although the appearance may be secondary to atelectasis, underlying pneumonic infiltration cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings. No mass lesion was observed in both lungs. There are a few millimetric nodules in the left lung, the largest of which is 5.5x3.5 mm in the upper lobe inferior lingular segment. Emphysematous changes were observed in both lungs. There are findings consistent with peritoneal carcinomatosis in the upper abdominal sections within the image, and hypodense lesions in the liver and spleen parenchyma that cannot be characterized in this examination. No lytic or destructive lesions were detected in the bone structures within the image."
impressions="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings.  A few millimetric nodules in the left lung."
description= "Pericardial effusion, Emphysema, Atelectasis, Lung nodule, Pleural effusion, Consolidation"
nifti_file = '../data/nifti/ct-rate/train/train_100_a_1.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_100_a_1/original/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description="", 
               modality = "CT",
               study_instance_uid='1.3.1',
               study_id='1', 
               accession_number='1.3',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,

               )

c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (90) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_100_a_1/original/
Deleted Study: 0f3fe58a-fdfc6ac0-0e8d76f1-cfcc5847-f520f5cb
Files Uploaded to Orthanc Server Localhost


### Generated

In [154]:
findings = "A central venous catheter inserted from the right was observed. Trachea and both main bronchi are open and no obstructive pathology is detected. Mediastinal vascular structures and cardiac examination could not be evaluated optimally due to the lack of contrast. The pulmonary trunk caliber was measured at 30 mm and was wider than normal. Heart contour, the size is natural. There is minimal pericardial effusion. The effusion measuring approximately 80 mm in size is observed on the right. There is an area of increased density in the lung parenchyma adjacent to the effusion, which is considered secondary to compressive atelectasis. No pathological increase in wall thickness was observed in the thoracic esophagus. In the mediastinum, in both axillary regions and in the supraclavicular fossa, no lymph nodes were observed in pathological size and appearance. In the lower lobe of the left lung, there is an area of increase in density consistent with the consolidation observed in the air bronchograms. Although the appearance may be secondary to atelectasis, underlying pneumonic infiltration cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings. No mass lesion was observed in both lungs. There are a few millimetric nodules in the left lung, the largest of which is 5.5x3.5 mm in the upper lobe inferior lingular segment. Emphysematous changes were observed in both lungs. There are findings consistent with peritoneal carcinomatosis in the upper abdominal sections within the image, and hypodense lesions in the liver and spleen parenchyma that cannot be characterized in this examination. No lytic or destructive lesions were detected in the bone structures within the image."
impressions="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings.  A few millimetric nodules in the left lung."
description= "Pericardial effusion, Emphysema, Atelectasis, Lung nodule, Pleural effusion, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_100_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_100_a_1/generated/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.3.2',
               study_id='2', 
               accession_number='1.3',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,
               )

c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (458) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_100_a_1/generated/
Deleted Study: da6aab70-a534ccc1-f2dbe1c6-56ee2620-67514332
Files Uploaded to Orthanc Server Localhost


# Example 4
- Lung nodule


### Original 

In [155]:
findings = "No lymph node was observed in the supraclavicular fossa and axilla in pathological size and appearance. Thyroid gland, left lobe and isthmus are operated. Heart dimensions and compartments appear natural. Pericardial effusion was not detected. Calibrations of mediastinal major vascular structures are natural. No pneumonic infiltration or consolidation area was detected in the lung parenchyma. No suspicious mass or nodular space-occupying lesion was observed in the lung parenchyma. There are a few nonspecific nodules less than 5 mm in diameter. In the evaluation of the upper abdominal organs, including the cross-section, there is a decrease in both kidney sizes and parenchyma thickness. No lytic-destructive lesions were detected in bone structures."
impressions="Thoracic CT examination within normal limits. Decrease in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/train/train_1000_a_1.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1000_a_1/original/'

nifti_to_dicom(nifti_file=nifti_file,
               output_folder=output_folder,
               series_description="", 
               modality = "CT",
               study_instance_uid='1.4.1',
               study_id='1', 
               accession_number='1.4',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description
               )

Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1000_a_1/original/
Deleted Study: 90b692ad-11a95190-eaaee368-2bee617f-3501974a
Files Uploaded to Orthanc Server Localhost


## Generated

In [156]:
findings = "No lymph node was observed in the supraclavicular fossa and axilla in pathological size and appearance. Thyroid gland, left lobe and isthmus are operated. Heart dimensions and compartments appear natural. Pericardial effusion was not detected. Calibrations of mediastinal major vascular structures are natural. No pneumonic infiltration or consolidation area was detected in the lung parenchyma. No suspicious mass or nodular space-occupying lesion was observed in the lung parenchyma. There are a few nonspecific nodules less than 5 mm in diameter. In the evaluation of the upper abdominal organs, including the cross-section, there is a decrease in both kidney sizes and parenchyma thickness. No lytic-destructive lesions were detected in bone structures."
impressions="Thoracic CT examination within normal limits. Decrease in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1000_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1000_a_1/generated/'

nifti_to_dicom(nifti_file=nifti_file,
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.4.2',
               study_id='2', 
               accession_number='1.4',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description,
               )

c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (128) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1000_a_1/generated/
Deleted Study: 823a3e2c-e9c2f62f-d46ea046-0b2c81b9-c222a0a5
Files Uploaded to Orthanc Server Localhost


# Example 5
- Arterial wall calcification
- Cardiomegaly
- Pericardial effusion
- Atelectasis
- Pleural effusion

### Original

In [157]:
findings = "The examination of the patient was evaluated by comparing it with the previous pulmonary CT angiography examination. The cardiothoracic ratio increased in favor of the heart. Minimal pericardial and bilateral pleural effusion are observed. Millimetric calcific plaques follow in the aorta. Trachea and both main bronchi are open. No occlusive pathology was detected in the trachea and both main bronchi. No pathologically enlarged lymph nodes were detected in the mediastinum and hilar regions.3.2021. There is an area of atelectasis accompanied by volume loss in the lingular segment of the left lung upper lobe. There are significant increases in interlobular septal thickness in the lower lobes of both lungs on the left. No mass was detected in both lungs. No pathological wall thickness increase was observed in the esophagus within the sections. In the upper abdominal organs within the sections, no mass with distinguishable borders was detected as far as it can be observed within the borders of non-enhanced CT. No lytic-destructive lesions were detected in the bone structures within the sections."
impressions="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Arterial wall calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/train/train_1001_a_1.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1001_a_1/original/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description="", 
               modality = "CT",
               study_instance_uid='1.5.1',
               study_id='1', 
               accession_number='1.5',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description= description,
               )



c:\Users\mot99\miniconda3\envs\ohif-python\Lib\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (94) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1/original/
Deleted Study: 6f92c59b-e099872d-4d3770e2-bc775099-05132126
Files Uploaded to Orthanc Server Localhost


### Generated

In [158]:
findings = "The examination of the patient was evaluated by comparing it with the previous pulmonary CT angiography examination. The cardiothoracic ratio increased in favor of the heart. Minimal pericardial and bilateral pleural effusion are observed. Millimetric calcific plaques follow in the aorta. Trachea and both main bronchi are open. No occlusive pathology was detected in the trachea and both main bronchi. No pathologically enlarged lymph nodes were detected in the mediastinum and hilar regions.3.2021. There is an area of atelectasis accompanied by volume loss in the lingular segment of the left lung upper lobe. There are significant increases in interlobular septal thickness in the lower lobes of both lungs on the left. No mass was detected in both lungs. No pathological wall thickness increase was observed in the esophagus within the sections. In the upper abdominal organs within the sections, no mass with distinguishable borders was detected as far as it can be observed within the borders of non-enhanced CT. No lytic-destructive lesions were detected in the bone structures within the sections."
impressions="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1001_a_1/generated/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.5.2',
               study_id='2', 
               accession_number='1.5',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )

Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1/generated/
Deleted Study: f1545596-4b1ac5b4-a7b8fd2d-d1787054-176c2477
Files Uploaded to Orthanc Server Localhost


### Counterfactual

In [159]:
impressions="Cardiomegaly, no pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1001_a_1_counterf_1/generated/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.5.3',
               study_id='3', 
               accession_number='1.5',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )

Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1_counterf_1/generated/
Deleted Study: 8e166ac9-01b7b40c-bdf43ede-a675ddf0-d497dd7c
Files Uploaded to Orthanc Server Localhost


In [160]:
impressions="Cardiomegaly, significant pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_2_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1001_a_1_counterf_2/generated/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.5.4',
               study_id='4', 
               accession_number='1.5',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )

Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1_counterf_2/generated/
Deleted Study: 17b7756e-ae05e7d6-0256b7b6-ba1aeeb7-c290c11b
Files Uploaded to Orthanc Server Localhost


# Example 6
- Arterial wall calcification
- Coronary artery wall calcification
- Hiatal hernia
- Atelectasis
- Lung opacity
- Pulmonary fibrotic sequela
- Peribronchial thickening
- Consolidation
- Bronchiectasis

### Original

In [161]:
findings = "Trachea was in the midline of both main bronchi and no obstructive pathology was observed in the lumen. The mediastinum could not be evaluated optimally in the non-contrast examination. As far as can be seen; mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Diffuse atherosclerotic wall calcifications were observed in the thoracic aorta and coronary arteries. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. Sliding type hiatal hernia was observed at the lower end of the esophagus. No pathologically enlarged lymph nodes were detected in the mediastinum. No lymph node was observed in pathological size and appearance in the supraclavicular and axillary fossa. When examined in the lung parenchyma window; In both lungs, ground-glass densities including central-peripheral, crazy paving pattern and consolidation areas showing signs of vascular enlargement were observed. The outlook is consistent with Covid-19 pneumonia. The right hemidiaphragm is elevated. Diffuse linear subsegmental atelectatic changes were observed in the middle and lower lobes of the right lung. There are segmental-subsegmental bronchiectatic changes and peribronchial thickening in both lungs. Sequelae thickening was observed in the posterocostal pleura in both hemithorax. No mass lesion with distinguishable borders was detected in both lungs. A hypodense lesion area of 21x20 mm was observed at the junction of segment 4A-8 at the level of the liver dome and could not be characterized in this examination. The gallbladder was not observed (operated). The right adrenal gland and right kidney were not observed (operated). Diffuse thickening was observed in the left adrenal gland. Within the sections, free fluid-pathological lymph node was not observed in the abdomen. Mild dextroscoliosis with left opening was observed at the thoracic level. At mid-thoracic level, a bridging spur formation is observed in the right anterolateral corners of the vertebrae. Vertebral corpus heights are preserved."
impressions="Diffuse atherosclerotic wall calcifications in the thoracic aorta and coronary arteries.  Hiatal hernia.  Findings consistent with Covid-19 pneumonia in the lung parenchyma.  Elevation in the right hemidiaphragm, linear subsegmental atelectatic changes in the middle and lower lobes of the right lung.  Segmentary-subsegmentary tubular bronchiectasis, peribronchial thickening in both lungs.  Sequela thickening of posterocostal pleura in both hemithorax.  Hypodense lesion at the level of the liver dome (segment 4A-8 junction), which cannot be characterized in this examination.  Diffuse thickening of the left adrenal gland.  Spur formations bridging each other at the mid-thoracic level and mild dextroscoliosis with a secondary left-facing opening"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/train/train_101_a_1.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_101_a_1/original/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description="", 
               modality = "CT",
               study_instance_uid='1.6.1',
               study_id='1', 
               accession_number='1.6',
               patient_name="Example Patient 6",
               patient_id="train_101_a_1",
               description=description,
               )

Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_101_a_1/original/
Deleted Study: 16e9f71b-6fc9830f-bb3c3ebc-0409d07e-63dcfc45
Files Uploaded to Orthanc Server Localhost


### Generated

In [162]:
findings = "Trachea was in the midline of both main bronchi and no obstructive pathology was observed in the lumen. The mediastinum could not be evaluated optimally in the non-contrast examination. As far as can be seen; mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Diffuse atherosclerotic wall calcifications were observed in the thoracic aorta and coronary arteries. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. Sliding type hiatal hernia was observed at the lower end of the esophagus. No pathologically enlarged lymph nodes were detected in the mediastinum. No lymph node was observed in pathological size and appearance in the supraclavicular and axillary fossa. When examined in the lung parenchyma window; In both lungs, ground-glass densities including central-peripheral, crazy paving pattern and consolidation areas showing signs of vascular enlargement were observed. The outlook is consistent with Covid-19 pneumonia. The right hemidiaphragm is elevated. Diffuse linear subsegmental atelectatic changes were observed in the middle and lower lobes of the right lung. There are segmental-subsegmental bronchiectatic changes and peribronchial thickening in both lungs. Sequelae thickening was observed in the posterocostal pleura in both hemithorax. No mass lesion with distinguishable borders was detected in both lungs. A hypodense lesion area of 21x20 mm was observed at the junction of segment 4A-8 at the level of the liver dome and could not be characterized in this examination. The gallbladder was not observed (operated). The right adrenal gland and right kidney were not observed (operated). Diffuse thickening was observed in the left adrenal gland. Within the sections, free fluid-pathological lymph node was not observed in the abdomen. Mild dextroscoliosis with left opening was observed at the thoracic level. At mid-thoracic level, a bridging spur formation is observed in the right anterolateral corners of the vertebrae. Vertebral corpus heights are preserved."
impressions="Diffuse atherosclerotic wall calcifications in the thoracic aorta and coronary arteries.  Hiatal hernia.  Findings consistent with Covid-19 pneumonia in the lung parenchyma.  Elevation in the right hemidiaphragm, linear subsegmental atelectatic changes in the middle and lower lobes of the right lung.  Segmentary-subsegmentary tubular bronchiectasis, peribronchial thickening in both lungs.  Sequela thickening of posterocostal pleura in both hemithorax.  Hypodense lesion at the level of the liver dome (segment 4A-8 junction), which cannot be characterized in this examination.  Diffuse thickening of the left adrenal gland.  Spur formations bridging each other at the mid-thoracic level and mild dextroscoliosis with a secondary left-facing opening"
description="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_101_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_101_a_1/generated/'
nifti_to_dicom(nifti_file=nifti_file, 
               output_folder=output_folder,
               series_description=impressions, 
               modality = "AI",
               study_instance_uid='1.6.2',
               study_id='2', 
               accession_number='1.6',
               patient_name="Example Patient 6",
               patient_id="train_101_a_1",
               description=description,

               )

Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_101_a_1/generated/
Deleted Study: 16ad0c46-a37caf53-f3025c09-e2f487d3-114db062
Files Uploaded to Orthanc Server Localhost
